# Extract, Transform, and Load (ETL) Lockbit data

The goal of this repository is to transform the Lockbit leaked data in a way that is usable and meaninful to use.

In [1]:
original_url = 'lockbitapyx2kr5b7ma7qn6ziwqgbrij2czhcbojuxmgnwpkgv2yx2yd.onion'
filepath = 'paneldb_dump.zip'

In [2]:
# !pip install pyarrow

In [3]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

## 1. Function to un-compress a file into a folder

In [4]:
import zipfile
import tarfile
import gzip
import bz2
import os

def uncompress_file(filepath, output_dir=None):
    if not output_dir:
        output_dir = os.path.splitext(filepath)[0]  # default: same name, no extension
    os.makedirs(output_dir, exist_ok=True)

    output_path = None  # Will store the final path where content is extracted

    if filepath.endswith('.zip'):
        with zipfile.ZipFile(filepath, 'r') as zf:
            zf.extractall(output_dir)
        output_path = output_dir

    elif filepath.endswith(('.tar', '.tar.gz', '.tgz', '.tar.bz2', '.tbz2')):
        with tarfile.open(filepath, 'r:*') as tf:
            tf.extractall(output_dir)
        output_path = output_dir

    elif filepath.endswith('.gz') and not filepath.endswith('.tar.gz'):
        with gzip.open(filepath, 'rb') as f_in:
            out_path = os.path.join(output_dir, os.path.basename(filepath[:-3]))
            with open(out_path, 'wb') as f_out:
                f_out.write(f_in.read())
            output_path = out_path

    elif filepath.endswith('.bz2') and not filepath.endswith('.tar.bz2'):
        with bz2.open(filepath, 'rb') as f_in:
            out_path = os.path.join(output_dir, os.path.basename(filepath[:-4]))
            with open(out_path, 'wb') as f_out:
                f_out.write(f_in.read())
            output_path = out_path

    else:
        print(f"Unsupported file type: {filepath}")
        return

    print(f"Uncompressed '{filepath}' to: {os.path.abspath(output_path)}")

Testing:

In [5]:
uncompress_file(filepath)

Uncompressed 'paneldb_dump.zip' to: /mnt/c/Users/j.santanna/Desktop/lockbit_leak/paneldb_dump


## 2. Getting an overview of the data 

In [6]:
import os

def list_files_with_size_and_lines(root_folder):
    file_info_list = []

    for dirpath, _, filenames in os.walk(root_folder):
        for filename in filenames:
            filepath = os.path.join(dirpath, filename)
            try:
                size_bytes = os.path.getsize(filepath)
                size_kb = round(size_bytes / 1024, 2)

                # Try reading as text to count lines
                try:
                    with open(filepath, 'r', encoding='utf-8') as f:
                        line_count = sum(1 for _ in f)
                except Exception:
                    line_count = 'N/A'

                # Extract file extension
                _, extension = os.path.splitext(filename)
                extension = extension.lower() if extension else 'N/A'

                file_info_list.append({
                    'path': dirpath,
                    'file_name': filename,
                    'extension': extension,
                    'size_kb': size_kb,
                    'lines': line_count
                })

            except Exception as e:
                print(f"⚠️ Error reading file '{filepath}': {e}")

    return file_info_list


Applying:

In [7]:
data_path = '/mnt/c/Users/j.santanna/Desktop/lockbit_leak/paneldb_dump'

import pandas as pd
df_data_overview = pd.DataFrame(list_files_with_size_and_lines(data_path))
df_data_overview

,path,file_name,extension,size_kb,lines
0,/mnt/c/Users/j.santanna/Desktop/lockbit_leak/paneldb_dump,paneldb_dump.sql,.sql,26977.4,114055


## Parsing this .sql file

In [10]:
filepath = 'paneldb_dump/paneldb_dump.sql'

In [12]:
import re
import json
import os
import pandas as pd

# Extract table names
with open(filepath, "r", encoding="utf-8") as f:
    sql_text = f.read()

pattern = r'CREATE TABLE\s+[`"]?(\w+)[`"]?\s*\('
table_list = re.findall(pattern, sql_text, re.IGNORECASE)

# Extract tables and columns
with open(filepath, "r", encoding="utf-8") as f:
    sql_text = f.read()

sql_text = re.sub(r'/\*.*?\*/', '', sql_text, flags=re.DOTALL)
sql_text = re.sub(r'--.*?$', '', sql_text, flags=re.MULTILINE)

table_pattern = re.compile(
    r'CREATE TABLE\s+[`"]?(\w+)[`"]?\s*\((.*?)\)\s*(?:ENGINE|COMMENT|CHARSET|ROW_FORMAT|;)',
    re.DOTALL | re.IGNORECASE
)

tables_columns_list = {}

for table_name, column_block in table_pattern.findall(sql_text):
    columns = []
    col_defs = []
    parens = 0
    current = ''
    for char in column_block:
        if char == '(':
            parens += 1
        elif char == ')':
            parens -= 1
        if char == ',' and parens == 0:
            col_defs.append(current.strip())
            current = ''
        else:
            current += char
    if current.strip():
        col_defs.append(current.strip())
    for col_def in col_defs:
        if re.match(r'^(PRIMARY|UNIQUE|KEY|CONSTRAINT|FOREIGN)', col_def.strip(), re.IGNORECASE):
            continue
        match = re.match(r'[`"]?(\w+)[`"]?\s+[a-zA-Z]', col_def)
        if match:
            columns.append(match.group(1))
    tables_columns_list[table_name] = columns

# Parse inserts
dataframes_manual = {}

for table in table_list:
    target_table = table

    raw_builds_blocks = []
    lines = sql_text.splitlines()
    current_block = []
    collecting = False

    for i, line in enumerate(lines):
        normalized = line.replace(" ", "").lower()
        if "insertinto" in normalized and target_table in normalized:
            if collecting and current_block:
                raw_builds_blocks.append("\n".join(current_block))
            current_block = [line]
            collecting = True
        elif collecting:
            current_block.append(line)
            if line.strip().endswith(";"):
                raw_builds_blocks.append("\n".join(current_block))
                current_block = []
                collecting = False

    if collecting and current_block:
        raw_builds_blocks.append("\n".join(current_block))

    all_builds_rows = []
    for block in raw_builds_blocks:
        value_start = block.find("VALUES")
        if value_start != -1:
            values_chunk = block[value_start + len("VALUES"):].strip().rstrip(";")

            value_block = values_chunk
            value_block = re.sub(r'^.*?VALUES\s*', '', value_block.strip(), flags=re.IGNORECASE).rstrip(';')
            rows = []
            current = ''
            level = 0
            in_quote = False
            escape = False
            for char in value_block:
                if escape:
                    current += char
                    escape = False
                elif char == '\\':
                    current += char
                    escape = True
                elif char == "'":
                    current += char
                    in_quote = not in_quote
                elif char == '(' and not in_quote:
                    if level == 0:
                        current = ''
                    level += 1
                    current += char
                elif char == ')' and not in_quote:
                    level -= 1
                    current += char
                    if level == 0:
                        rows.append(current.strip())
                else:
                    current += char

            for row_str in rows:
                row_str = row_str.strip()[1:-1]
                fields = []
                current = ''
                in_quote = False
                escape = False
                for char in row_str:
                    if escape:
                        current += char
                        escape = False
                    elif char == '\\':
                        current += char
                        escape = True
                    elif char == "'":
                        current += char
                        in_quote = not in_quote
                    elif char == ',' and not in_quote:
                        fields.append(current.strip())
                        current = ''
                    else:
                        current += char
                fields.append(current.strip())

                def clean(val):
                    if val == 'NULL':
                        return None
                    if val.startswith("'") and val.endswith("'"):
                        raw = val[1:-1].replace("\\'", "'").replace('\\"', '"').replace('\\\\', '\\')
                        try:
                            return json.loads(raw)
                        except:
                            return raw
                    return val

                all_builds_rows.append(tuple(clean(f) for f in fields))

    columns = tables_columns_list.get(target_table)
    
    valid_rows = [row for row in all_builds_rows if len(row) == len(columns)]
    if columns and valid_rows:
        df_builds = pd.DataFrame(valid_rows, columns=columns)
    else:
        df_builds = pd.DataFrame(all_builds_rows)

    dataframes_manual[target_table] = df_builds

# Export DataFrames
os.makedirs("raw_parquet", exist_ok=True)

summary_tables_records = []

for name, df in dataframes_manual.items():
    globals()[f'df_{name}'] = df

    for col in df.select_dtypes(include=["object"]).columns:
        try:
            df[col] = df[col].astype("string[pyarrow]")
        except Exception:
            df[col] = df[col].astype(str)

    try:
        parquet_path = f"raw_parquet/df_{name}.parquet"
        df.to_parquet(parquet_path, index=False)
        print(f"Exported df_{name} with {len(df)} records")
        display(df.head(2))
        display(df.tail(2))
        summary_tables_records.append([f'df_{name}', len(df)])
    except Exception as e:
        print(f"Could not export df_{name}: {e}")
        summary_tables_records.append([f'df_{name}', f"ERROR: {e}"])


Exported df_api_history with 0 records


""


""


Exported df_btc_addresses with 59975 records


,id,type,target_id,advid,address
0,3,25,24,10,bc1q5979ns9zrhtq8j2znkf59ldmd4nxc4hamuq8ut
1,4,25,44,27,bc1qj4wvkmggfdpa60tk7ag75zd4yjr77zqa0r69yq


...

,id,parentid,login,password,is_admin,level,session_id,linesxi_on,reg_date,last_online,...,keep_messages_unread,toxid,toxdata,ips,permissions,paranoid_mode,created_at,updated_at,tag,invite_id
0,1,0,admin,gyCuCzeMxHpT7a5fSYRDY5DxzefrdD5g5ktiibyE,1,4,t2km45gl7n6njiapm846jsr0af23ps2l7nf618v0,1,1605453973,1745930762,...,0,A1A6D2ECC8DB18DA0D5F04C5ED01A565B5A46E4012FAE627ACCB5D709BB89477D26BE7EF852C,<NA>,<NA>,"['chats', 'write_chat', 'news', 'faq', 'lb_green', 'lb_black', 'builder_chat_generate', 'linesxi', 'builder_language_check', 'lbb_without_password']",1,31/01/2022 22:20,29/04/2025 12:46,,0
1,2,0,matrix777,81jlXQFHOdGiY8C3peSZxFUxtUKjbIhh1936k6xo,0,1,1jjhf6np9a0mk680n1gik5k99shbgjml0nfbs6et,1,1605453973,1745926182,...,0,A1A6D2ECC8DB18DA0D5F04C5ED01A565B5A46E4012FAE627ACCB5D709BB89477D26BE7EF852C,<NA>,<NA>,"['chats', 'write_chat', 'news', 'faq', 'lb_green', 'lb_black', 'builder_chat_generate', 'linesxi', 'builder_language_check', 'lbb_without_password']",1,31/01/2022 22:20,29/04/2025 11:29,,0


,id,parentid,login,password,is_admin,level,session_id,linesxi_on,reg_date,last_online,...,keep_messages_unread,toxid,toxdata,ips,permissions,paranoid_mode,created_at,updated_at,tag,invite_id
73,75,0,ExtortionLord,gRh7539syEBj0lypdgi5,0,1,eqndti1cu29u6j0668setr7m44tg9noqmua36ik2,0,1745239132,1745854002,...,0,4DEBEB92B3A547C9F0E563CA015D725D76FE47C30E482B1C42C363D10394160D93AA54DB55A9,<NA>,<NA>,"['chats', 'write_chat', 'news', 'faq', 'lb_green', 'lb_black', 'builder_chat_generate', 'linesxi', 'builder_language_check', 'lbb_without_password']",0,21/04/2025 12:38,28/04/2025 15:26,newbie,3388
74,76,0,wlteaml,v0NF9rJtCbPjy4lDh8Bn,0,1,f26137j7c4ek4evmri3c29vo9g34gkaphirleioc,0,1745317692,1745321432,...,0,3DCE1C43491FC92EA7010322040B254FDD2731001C2DDC2B9E819F0C946BDC3CD251FA3B694A,<NA>,<NA>,"['chats', 'write_chat', 'news', 'faq', 'lb_green', 'lb_black', 'builder_chat_generate', 'linesxi', 'builder_language_check', 'lbb_without_password']",0,22/04/2025 10:28,22/04/2025 11:30,newbie,3411


Exported df_visits with 2398 records


,id,clientid,visit_date,created_at
0,46,24,1734583516,2024-12-19 04:45:16
1,47,24,1734586364,2024-12-19 05:32:44


,id,clientid,visit_date,created_at
2396,2461,263,1745930430,2025-04-29 12:40:30
2397,2462,277,1745930964,2025-04-29 12:49:24


# Now that everything is parsed this is how you can load each .parquet 

In [20]:
import pandas as pd

df_api_history = pd.read_parquet("raw_parquet/df_api_history.parquet")
df_btc_addresses = pd.read_parquet("raw_parquet/df_btc_addresses.parquet")
df_builds = pd.read_parquet("raw_parquet/df_builds.parquet")
df_builds_configurations = pd.read_parquet("raw_parquet/df_builds_configurations.parquet")
df_chats = pd.read_parquet("raw_parquet/df_chats.parquet")
df_clients = pd.read_parquet("raw_parquet/df_clients.parquet")
df_events = pd.read_parquet("raw_parquet/df_events.parquet")
df_events_seen = pd.read_parquet("raw_parquet/df_events_seen.parquet")
df_faq = pd.read_parquet("raw_parquet/df_faq.parquet")
df_files = pd.read_parquet("raw_parquet/df_files.parquet")
df_invites = pd.read_parquet("raw_parquet/df_invites.parquet")
df_jobs = pd.read_parquet("raw_parquet/df_jobs.parquet")
df_migrations = pd.read_parquet("raw_parquet/df_migrations.parquet")
df_news = pd.read_parquet("raw_parquet/df_news.parquet")
df_pkeys = pd.read_parquet("raw_parquet/df_pkeys.parquet")
df_socket_messages = pd.read_parquet("raw_parquet/df_socket_messages.parquet")
df_system_invalid_requests = pd.read_parquet("raw_parquet/df_system_invalid_requests.parquet")
df_testfiles = pd.read_parquet("raw_parquet/df_testfiles.parquet")
df_users = pd.read_parquet("raw_parquet/df_users.parquet")
df_visits = pd.read_parquet("raw_parquet/df_visits.parquet")